In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision import transforms
import time
from tqdm.autonotebook import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from inspect import getfullargspec,signature

import matplotlib.pyplot as plt
import numpy as np

from torch.utils.data import Dataset
from torchvision import datasets

import torch as ch
from robustness.datasets import FashionMnist
ds = FashionMnist('/tmp/FashionMNIST')

%matplotlib inline
%config InlineBackend.figure_format = 'svg'


In [2]:
############################
class MnistResNet(nn.Module):
  def __init__(self, in_channels=1):
    super(MnistResNet, self).__init__()

    # Load a pretrained resnet model from torchvision.models in Pytorch
    self.model = models.resnet50(pretrained=False)

    # Change the input layer to take Grayscale image, instead of RGB images.  **************
    # Hence in_channels is set as 1 or 3 respectively
    # original definition of the first layer on the ResNet class
    # self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
    self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
    
    # Change the output layer to output 10 classes instead of 1000 classes
    num_ftrs = self.model.fc.in_features
    self.model.fc = nn.Linear(num_ftrs, 10)

  def forward(self, x):
    return self.model(x)


my_resnet = MnistResNet()

input = ch.randn((16,1,244,244))
output = my_resnet(input)
print(output.shape)

print(my_resnet)

torch.Size([16, 10])
MnistResNet(
  (model): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample):

In [3]:
        
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")
    
# load STD model 
model_std = MnistResNet().to(device)

# params you need to specify:
epochs = 150
batch_size = 128
model_std = MnistResNet()
model_state_dict = torch.load("./FashionMnistSTD50Net")
model_std.load_state_dict(model_state_dict)
#model_std.to(device)

<All keys matched successfully>

In [4]:
### Initiate ADV model 
from robustness.model_utils import make_and_restore_model
model_adv, _ = make_and_restore_model(arch='resnet50', dataset=ds,
              resume_path= '/home/sharim.jamal/FashionMnistAdv50/checkpoint.pt.latest') #'/tmp/35daedae-1b39-4941-ad08-8bd6459c1bd8/checkpoint.pt.best')
model_adv.eval()
pass

=> loading checkpoint '/home/sharim.jamal/FashionMnistAdv50/checkpoint.pt.latest'
=> loaded checkpoint '/home/sharim.jamal/FashionMnistAdv50/checkpoint.pt.latest' (epoch 150)


In [5]:
#only load the validation set
ATTACK_EPS = 0.5
ATTACK_STEPSIZE = 0.1
ATTACK_STEPS = 10
NUM_WORKERS = 8
BATCH_SIZE = 128

kwargs = {
    'constraint':'2', # use L2-PGD
    'eps': ATTACK_EPS, # L2 radius around original image
    'step_size': ATTACK_STEPSIZE,
    'iterations': ATTACK_STEPS,
    'do_tqdm': True,
}
#label.size()
from sklearn import metrics

# TensorDataset loaders
_, test_loader = ds.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE) #make it bigger

test_enum = enumerate(test_loader)
# measure accuracy on the validation set but in batch sizes, since robust model can't memory-handle large image sizes at once
acc_list = []

for _ in range(len(test_loader)):
    _, (im, label) = next(test_enum)  
    im,label = im.to(device),label.to(device)
    
    #the model can generate adv by applying max l-2 loss
    _, im_adv = model_adv(im, label, make_adv=True, **kwargs)
    im_adv_transformed = transforms.Resize((224, 224)).forward(im_adv)
    
    
    im_adv = im_adv.to(device='cpu')
    im_adv_transformed = im_adv_transformed.to(device='cpu')
    
    # Get predicted labels for adversarial examples using 
    pred = model_std(im_adv_transformed)
    label_pred = ch.argmax(pred, dim=1)
    
    im,label = im.to(device='cpu'),label.to(device='cpu')
    im_adv = im_adv.to(device='cpu')
    im_adv_transformed = im_adv_transformed.to(device='cpu')
    
    #Get accuracy
    acc_list.append(metrics.accuracy_score(label ,label_pred.cpu()))
    print("Accuracy:",metrics.accuracy_score(label ,label_pred.cpu()))
    
    #test_list = torch.cat((test_list,im_adv_transformed.to(device='cpu')),dim=0)
    #label_list = torch.cat((label_list,label),dim=0)

==> Preparing dataset fashionmnist..


/home/sharim.jamal/.conda/envs/test/lib/python3.7/site-packages/torchvision/datasets/mnist.py:62: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
Current loss: 0.3595261573791504: 100%|█████████| 10/10 [00:02<00:00,  4.28it/s]


Accuracy: 0.6171875


Current loss: 0.6930171251296997: 100%|█████████| 10/10 [00:01<00:00,  6.10it/s]


Accuracy: 0.6640625


...

Accuracy: 0.7265625


Current loss: 0.7046204805374146: 100%|█████████| 10/10 [00:01<00:00,  5.84it/s]


Accuracy: 0.671875


Current loss: 1.042877197265625: 100%|██████████| 10/10 [00:00<00:00, 19.35it/s]


Accuracy: 0.6875


In [7]:
print(torch.cuda.is_available())

True


In [8]:
## Generate adv examples 

print("Average over all batches ",   sum(acc_list) / len(acc_list))   

#im_adv_transformed = transforms.Resize((224, 224)).forward(im_adv)
#print(im_adv[0].size())
#print(im_adv_transformed[0].size())
# predict and measure the accuracy 

# Get predicted labels for adversarial examples using 
#pred = model_std(test_list)
#label_pred = ch.argmax(pred, dim=1)


Average over all batches  0.6654469936708861
